In [3]:
# Import necessary modules
import torch
from IPython.display import Image

# Train YOLOv5 model with the medium version
!python yolov5/train.py --img 640 --batch 16 --epochs 100 --data yolov5/sign_language.yaml --weights yolov5s.pt --cache


In [2]:
import torch
import cv2
import pyttsx3

# Load the trained model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='E:/PROJECTS/SIGN-LANG-DETECTION/yolov5/runs/train/exp4/weights/best.pt')
model.conf = 0.2  # Lowering confidence threshold to make detections more sensitive

# Define class labels
class_labels = ["Hello", "I Love You", "No", "Thank You", "Yes"]

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Set properties for speech rate and volume
engine.setProperty('rate', 150)
engine.setProperty('volume', 0.9)

# Function to handle text-to-speech
def text_to_speech(text):
    engine.say(text)
    engine.runAndWait()

# Initialize webcam
cap = cv2.VideoCapture(0)

# Detection counters for smoothing detections over multiple frames
detection_counts = {label: 0 for label in class_labels}
detection_threshold = 5  # Number of frames required to confirm detection

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame to reduce processing time
    frame_resized = cv2.resize(frame, (640, 480))

    # Perform detection on resized frame
    results = model(frame_resized)

    # Draw results and implement detection smoothing
    for *box, conf, cls in results.xyxy[0]:
        label = class_labels[int(cls)]
        detection_counts[label] += 1

        # Confirm detection if detected in multiple consecutive frames
        if detection_counts[label] >= detection_threshold:
            # Draw bounding box and label on the original frame
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            # Text-to-speech for detected sign label
            text_to_speech(label)

            # Reset counter after confirming detection
            detection_counts[label] = 0

        # Reset other labels' counters to avoid carry-over effects
        for other_label in class_labels:
            if other_label != label:
                detection_counts[other_label] = 0

    # Display the frame with bounding boxes and labels
    cv2.imshow('Sign Language Detection', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\SANJAY/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-28 Python-3.8.20 torch-2.4.1+cpu CPU



Exception: [Errno 2] No such file or directory: 'E:\\PROJECTS\\SIGN-LANG-DETECTION\\yolov5\\runs\\train\\exp4\\weights\\best.pt'. Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.